In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import os

# ================= 1. Read DOE data =================
base_path = r"D:\1015 model DB"
doe_file = os.path.join(base_path, "DOE_simulated_BCHS_1h_fromEPW.csv")

data = pd.read_csv(doe_file, parse_dates=['Timestamp'], index_col='Timestamp')

# 添加时间特征
data['hour'] = data.index.hour
data['dayofweek'] = data.index.dayofweek

# ================= 2. Define objectives and characteristics =================
target_col = 'Total electricity consumption' 
feature_cols = ['Total electricity consumption', 'BCHS PV Energy', 'Wall_UValue', 'hour', 'dayofweek']

X = data[feature_cols]
y = data[target_col]  

# ================= 3. Split into training/test sets =================
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# ================= 4. Training a Random Forest Model =================
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)
print(f"Model training completed, R² score is reasonable.")

# ================= 5. PEMFC + Battery parameters =================
time_step_hours = 1
pemfc_power_max = 200
pemfc_efficiency = 0.6
hydrogen_LHV = 33.3  # kWh/kg
battery_capacity_kWh = 2000
battery_power_max = 250
charge_efficiency = 0.95
discharge_efficiency = 0.95
soc_min, soc_max = 0.3, 1.0
total_h2_available = 6 * 200 

battery_soc_current = X_test.iloc[0]['Wall_UValue']  # 这里假设初始SOC与墙体U值对应，可根据需求修改

# ================= 6. Simulation Loop =================
records = []

for idx, row in X_test.iterrows():
    load_kw = row['Total electricity consumption']
    pv_kw = row['BCHS PV Energy']
    battery_soc = battery_soc_current
    hour = row['hour']
    dayofweek = row['dayofweek']

    remaining_load = load_kw - pv_kw

    # Predicting PEMFC Output
    input_features = pd.DataFrame([[load_kw, pv_kw, battery_soc, hour, dayofweek]], columns=feature_cols)
    pemfc_power_pred = model.predict(input_features)[0]
    pemfc_power = np.clip(pemfc_power_pred, 0, pemfc_power_max)

    # Hydrogen Consumption Calculation
    if total_h2_available > 0:
        h2_needed = pemfc_power * time_step_hours / (pemfc_efficiency * hydrogen_LHV)
        if h2_needed > total_h2_available:
            pemfc_power = total_h2_available * pemfc_efficiency * hydrogen_LHV / time_step_hours
            h2_needed = total_h2_available
        total_h2_available -= h2_needed
        total_h2_available = max(total_h2_available, 0)
    else:
        pemfc_power = 0
        h2_needed = 0

    remaining_load -= pemfc_power

    # Battery discharge supplementary load
    if remaining_load > 0:
        available_energy = (battery_soc_current - soc_min) * battery_capacity_kWh
        discharge_power = min(remaining_load, battery_power_max, available_energy)
        delta_soc = discharge_power / battery_capacity_kWh / discharge_efficiency
        battery_soc_current = max(battery_soc_current - delta_soc, soc_min)
        battery_power = discharge_power
        remaining_load -= discharge_power
    else:
        battery_power = 0

    # PV充电电池
    if remaining_load < 0:
        charge_power = -remaining_load
        space = battery_capacity_kWh * (soc_max - battery_soc_current)
        charge_power = min(charge_power, battery_power_max, space)
        delta_soc = charge_power / battery_capacity_kWh * charge_efficiency
        battery_soc_current = min(battery_soc_current + delta_soc, soc_max)
        battery_power = -charge_power
        remaining_load = 0

    # ower Grid Supplement
    grid_power = max(remaining_load, 0)

    records.append({
        'Timestamp': idx,
        'Load (kW)': load_kw,
        'PV Power (kW)': pv_kw,
        'PEMFC Power (kW)': pemfc_power,
        'Battery Power (kW)': battery_power,
        'Grid Power (kW)': grid_power,
        'Battery SOC': battery_soc_current,
        'H2 Remaining (kg)': total_h2_available
    })


df_results['PEMFC Power Smoothed'] = df_results['PEMFC Power (kW)'].rolling(window=24, min_periods=1).mean()  # 24小时滑动平均

# MONTH-AVERAGED DATA
df_monthly = df_results.resample('M').mean()
df_monthly['PEMFC Power Smoothed'] = df_monthly['PEMFC Power Smoothed'].ewm(span=3, adjust=False).mean()
df_monthly['H2 Remaining Smoothed'] = df_monthly['H2 Remaining (kg)'].ewm(span=3, adjust=False).mean()
df_monthly['Battery SOC Smoothed'] = df_monthly['Battery SOC'].ewm(span=3, adjust=False).mean()

# ================= 7. 可视化 =================
fig, axes = plt.subplots(1, 2, figsize=(14, 4), sharex=False)

# (a) Load and Power Grid
axes[0].plot(df_results.index, df_results['Load (kW)'], label='Original Load', color='#c2768b', linewidth=1.2)
axes[0].plot(df_results.index, df_results['Grid Power (kW)'], label='Residual Load', color='#71b7ed', linestyle='--', linewidth=1)
axes[0].set_ylabel('Power (kW)')
axes[0].set_title('(a) Load and Grid Power')
axes[0].legend(loc='upper right', frameon=False)
axes[0].grid(True, linestyle='--', alpha=0.6)

# (b) PEMFC + SOC + H2
ax1 = axes[1]
ax2 = ax1.twinx()
ax1.plot(df_monthly.index, df_monthly['PEMFC Power Smoothed'], label='PEMFC Output Power', color='#84c3b7', linewidth=1.5, marker='o')
ax1.plot(df_monthly.index, df_monthly['H2 Remaining Smoothed'], label='H2 Remaining', color='#f57c6e', linewidth=1.5, marker='s')
ax2.plot(df_monthly.index, df_monthly['Battery SOC Smoothed'], label='Battery SOC', color='#b8aeeb', linewidth=1.5, marker='^')
ax1.set_ylabel('PEMFC Power (kW) / H2 (kg)')
ax2.set_ylabel('Battery SOC')
ax1.set_title('(b) PEMFC Output, Battery SOC and H2 Remaining (Monthly Avg)')

lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper right', frameon=False, fontsize=9)
ax1.grid(True, linestyle='--', alpha=0.6)

plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

def select_time_range(dt):
    hour = dt.hour
    return (hour >= 23) or (hour <= 14)

df_filtered = df_results[df_results.index.map(select_time_range)].copy()

# Define the start of each day as 23:00 on the previous day.
df_filtered['custom_date'] = (df_filtered.index - pd.Timedelta(hours=23)).date
df_filtered['custom_date'] = pd.to_datetime(df_filtered['custom_date'])

# ================= 2. Aggregate Daily Energy Supply =================
battery_discharge = df_filtered['Battery Power (kW)'].apply(lambda x: x if x > 0 else 0)
pemfc = df_filtered['PEMFC Power (kW)']
pv = df_filtered['PV Power (kW)']

pemfc_batt_sum = pemfc.groupby(df_filtered['custom_date']).sum() + \
                 battery_discharge.groupby(df_filtered['custom_date']).sum()
pv_sum = pv.groupby(df_filtered['custom_date']).sum()

df_daily = pd.DataFrame({
    'PEMFC + Battery Supply': pemfc_batt_sum,
    'Total Supply (with PV)': pemfc_batt_sum + pv_sum
})

dates = df_daily.index
x = np.arange(len(dates))

# ================= 3. pictures =================
fig, ax = plt.subplots(figsize=(14, 4))

# PEMFC + Battery
ax.bar(x, df_daily['PEMFC + Battery Supply'], label='PEMFC + Battery Supply', color='#b7282e')

# PV
ax.bar(x, df_daily['Total Supply (with PV)'] - df_daily['PEMFC + Battery Supply'],
       bottom=df_daily['PEMFC + Battery Supply'],
       label='PV Supply', color='#313772', alpha=0.7)

# X-axis
target_labels = ['2020-01', '2020-03', '2020-05', '2020-07', '2020-09', '2020-11', '2021-01']
target_dates = pd.to_datetime(target_labels, format='%Y-%m')
tick_positions = [dates.get_indexer([td], method='nearest')[0] for td in target_dates]

ax.set_xticks(tick_positions)
ax.set_xticklabels(target_labels, rotation=0)

ax.set_title('Daily Energy Supply Composition (23:00 to next day 14:00)')
ax.set_ylabel('Energy (kWh)')
ax.set_xlabel('Date')
ax.legend()
ax.grid(True, linestyle='--', alpha=0.5)
plt.tight_layout()
plt.show()

# ================= 4. pictures=================
denominator = pv + pemfc + battery_discharge
denominator = denominator.where(denominator > 1e-3, np.nan)  # 避免除0

supply_ratio = (pemfc + battery_discharge) / denominator * 100
supply_ratio = supply_ratio.fillna(0)
supply_ratio_smooth = supply_ratio.rolling(window=5, min_periods=1).mean()

plt.figure(figsize=(14, 4))
plt.plot(df_filtered.index, supply_ratio_smooth, color='#4D4D9F', alpha=0.6, label='PEMFC + Battery Contribution')
plt.ylim(0, 105)
plt.title('Percentage Contribution of PEMFC + Battery Discharge (23:00 to next day 14:00)')
plt.ylabel('Contribution (%)')
plt.xlabel('Time')
plt.legend()
plt.grid(True, linestyle='--', alpha=0.5)
plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import numpy as np

# -----------------------------
# 1. Data path
# -----------------------------
base_path = r"D:\1015 model DB"
static_file = f"{base_path}\\DOE_static_estimate.csv"
construction_file = f"{base_path}\\NewYorkconstruction_summary.csv"

static = pd.read_csv(static_file)
construction = pd.read_csv(construction_file)

# -----------------------------
# 2. Baseline Model Load (provided)
# -----------------------------
Q_total = static["TotalHeatLoad_W"]
E_baseline = static["Original electricity consumption"]  # kWh

# -----------------------------
# 3. Estimating the Energy Consumption of Ground-Source Heat Pumps
# -----------------------------
T_out = static["Outside Dry-Bulb Temperature"]

COP_heating = np.clip(3.5 - 0.05*(20 - T_out), 2.0, 4.5)   # 简化假设
COP_cooling = np.clip(4.0 - 0.03*(T_out - 25), 2.5, 5.0)

# Distinguish between positive and negative loads (heating/cooling)
Q_heat = Q_total.clip(lower=0)
Q_cool = (-Q_total).clip(lower=0)

# Calculate the power consumption of the heat pump (W)
E_GSHP_W = Q_heat / COP_heating + Q_cool / COP_cooling

# changed to kWh
E_GSHP_kWh = E_GSHP_W / 1000.0

# -----------------------------
# 4. Energy Consumption vs. Carbon Reduction
# -----------------------------
static["GSHP electricity consumption (kWh)"] = E_GSHP_kWh
static["Electricity saving (kWh)"] = static["Original electricity consumption"] - E_GSHP_kWh

# Annual Energy Savings Rate
annual_saving_rate = static["Electricity saving (kWh)"].sum() / static["Original electricity consumption"].sum()

print(f"🏠 年度节能率约为：{annual_saving_rate:.2%}")

# -----------------------------
# 5. Carbon Emission Reduction Estimate (Assuming U.S. Grid Emission Factor of 0.35 kgCO2/kWh)
# -----------------------------
CO2_factor = 0.35  # kgCO2/kWh
CO2_saving = static["Electricity saving (kWh)"].sum() * CO2_factor
print(f"🌿 年碳减排约为：{CO2_saving/1000:.1f} t CO2")

# -----------------------------
# 6. Results
# -----------------------------
out_file = f"{base_path}\\DOE_GSHP_comparison.csv"
static.to_csv(out_file, index=False)
print(f"✅ saved into：{out_file}")

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

# -----------------------------
# 1. Data
# -----------------------------
INPUT_CSV = r"D:\1015 model DB\DOE_GSHP_comparison.csv"
df = pd.read_csv(INPUT_CSV)
df["Timestamp"] = pd.to_datetime(df["Timestamp"])

# -----------------------------
# 2. Energy-saving calculation
# -----------------------------
df["Saving_kWh"] = df["Original electricity consumption"] - df["GSHP electricity consumption (kWh)"]

total_original = df["Original electricity consumption"].sum()
total_gshp = df["GSHP electricity consumption (kWh)"].sum()
total_saving = df["Saving_kWh"].sum()
saving_rate = (total_saving / total_original) * 100

# -----------------------------
# 3. Calculate carbon emissions
# -----------------------------
EF = 0.4  # kg CO2 per kWh
CO2_original = total_original * EF
CO2_GSHP = total_gshp * EF
CO2_saving = total_saving * EF

# -----------------------------
# 4. Picrtures
# -----------------------------
fig, ax1 = plt.subplots(figsize=(16, 7))

# left-axis
ax1.plot(df["Timestamp"], df["Original electricity consumption"], label="Original Electricity", color="#6656A6", alpha=0.8, linewidth=1)
ax1.plot(df["Timestamp"], df["GSHP electricity consumption (kWh)"], label="GSHP Electricity", color="#e76254", alpha=0.8, linewidth=1)
ax1.bar(df["Timestamp"], df["Saving_kWh"], label="Hourly Saving", color="#ffe6b7", alpha=0.4, width=1.5)

ax1.set_xlabel("Month")
ax1.set_ylabel("Hourly Electricity (kWh)", color="#6656A6")
ax1.tick_params(axis='y', labelcolor="#6656A6")
ax1.xaxis.set_major_locator(mdates.MonthLocator())
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%b'))
ax1.grid(True, linestyle='--', alpha=0.3)

# small-picture
axins = inset_axes(ax1, width="25%", height="40%", loc='upper center')
axins.bar(['Original', 'GSHP', 'Saved'], [CO2_original, CO2_GSHP, CO2_saving],
          color=['tab:red', 'tab:blue', 'tab:green'], alpha=0.4)
axins.set_ylabel("CO2 (kg)")
axins.set_title("Annual CO2 Emission & Saving", fontsize=10, pad=-10)

# tittles
ax1.legend(fontsize=12, loc="upper left")
plt.suptitle("Electricity Consumption & Saving Trend", fontsize=16)
plt.tight_layout()
plt.show()

# -----------------------------
# 5. print
# -----------------------------
print("\n📊 Energy consumption summary results:：")
print(f"riginal System Total Electricity Consumption: {total_original:.2f} kWh")
print(f"GSHP System Total Electricity Consumption:  {total_gshp:.2f} kWh")
print(f"Total Electricity Savings: {total_saving:.2f} kWh")
print(f"Energy Savings Rate: {saving_rate:.2f}%")

print("\n🌱 Carbon Emissions Summary Results:")
print(f"Original System Total Carbon Emissions:{CO2_original:.2f} kg CO2")
print(f"Original System Total Carbon Emissions: {CO2_GSHP:.2f} kg CO2")
print(f"Carbon Emissions Saved:  {CO2_saving:.2f} kg CO2")